In [ ]:
# Install necessary libraries
!pip install -q torch torchvision timm mlflow pyngrok pytorch-lightning==2.4.0
!pip install -q "huggingface-hub<1.0" "transformers<4.44" torchmetrics==1.4.0 scikit-learn numpy matplotlib seaborn

# CLEAN CACHE (optional but recommended for fresh installs/updates)
import os, shutil
if os.path.exists("/root/.cache/huggingface/hub"):
    shutil.rmtree("/root/.cache/huggingface/hub")

# ===============================
# 📁 System & File Handling
# ===============================
import os                    # OS operations (paths, env vars)
import shutil                # High-level file operations
from pathlib import Path     # Clean path handling

# ===============================
# 🖼 Image Processing
# ===============================
from PIL import Image        # Image loading & processing

# ===============================
# 🔥 PyTorch Core
# ===============================
import torch                 # Core PyTorch
from torch.utils.data import DataLoader

# ===============================
# ⚡ PyTorch Lightning
# ===============================
import pytorch_lightning as pl

# ===============================
# 🧠 Vision & Transforms
# ===============================
from torchvision import datasets, transforms
import timm                  # Pretrained vision models (EfficientNet, ViT, ConvNeXt)

# ===============================
# 📊 Experiment Tracking (MLflow)
# ===============================
import mlflow
import mlflow.pytorch

# ===============================
# 🌐 Public MLflow UI (ngrok)
# ===============================
from pyngrok import ngrok

In [ ]:
# This line checks whether a GPU (CUDA) is available.
# If a GPU exists, training will run on the GPU for much faster performance.
# If no GPU is found (e.g., running on CPU-only machine), it will fall back to CPU.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Print which device the model will use
print("Running on:", device)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# =============================
# 🔧 USER CONFIGURATION
# (Edit these values according to your project)
# =============================

# 👉 Your Ngrok authentication token (required to create a public URL for MLflow UI)
# ⚠️ Keep this private. Anyone with this token can use your ngrok account.
NGROK_AUTH_TOKEN = "36CLviJ3eedsGnzgPHm7MM9g4De_4tDSxH3CgGEQCytxcXKgX"

# 👉 Directory where MLflow will store run logs, metrics, parameters, and model artifacts
MLRUNS_DIR = "/content/drive/MyDrive/BongoDev/Project-1/mlruns"

# 👉 Path to your dataset ZIP file (usually stored in Google Drive)
# ML pipeline will unzip and prepare the dataset from here.
DATA_ZIP = "/content/drive/MyDrive/BongoDev/Project-1/archive.zip"

# 👉 Directory where the dataset should be extracted
EXTRACT_DIR = "/content/drive/MyDrive/BongoDev/Project-1/Potato"

# 👉 Image size for training (input size for vision models like ViT, EfficientNet, ResNet)
IMG_SIZE = 224

# 👉 Number of images per batch during training
# Higher batch size → faster training but requires more GPU memory
BATCH_SIZE = 16

# 👉 Maximum number of epochs the model will train
# One epoch = one full pass over the entire dataset
MAX_EPOCHS = 2


In [ ]:
from pyngrok import ngrok
# pyngrok → a Python wrapper for ngrok, allowing you to create public URLs
#            for local services (like MLflow UI) directly from code.

ngrok.kill()
# This command closes ALL existing ngrok tunnels.
# Why?
# - Prevents "address already in use" errors
# - Ensures a clean state before creating a new ngrok tunnel
# - Useful in Google Colab where old tunnels may stay alive across runs

In [ ]:
def start_mlflow_ui(port: int = 5000, mlruns_dir: str = MLRUNS_DIR):

    # -----------------------------------------
    # Create the MLflow directory if not exists
    # -----------------------------------------
    os.makedirs(mlruns_dir, exist_ok=True)
    # MLflow stores metrics, params, and artifacts under this folder.


    # -----------------------------------------
    # Kill previous MLflow + ngrok processes
    # Ensures a clean start (very important in Colab)
    # -----------------------------------------
    try:
        get_ipython().system_raw("pkill -f mlflow")
        # Kills any process related to "mlflow ui" running in the background
    except:
        pass

    try:
        ngrok.kill()
        # Closes ALL running ngrok tunnels, preventing port conflicts
    except:
        pass


    # -----------------------------------------
    # Add your NGROK auth token (required for tunnels)
    # -----------------------------------------
    os.system(f"ngrok config add-authtoken {NGROK_AUTH_TOKEN}")
    # Registers your ngrok token so tunnels work under your account.


    # -----------------------------------------
    # FIX 1: Host header rewrite
    # Required because MLflow runs on a different internal URL.
    # This prevents 502 Bad Gateway errors.
    # -----------------------------------------
    os.system("mkdir -p /root/.ngrok2")
    os.system("echo 'host_header: rewrite' >> /root/.ngrok2/ngrok.yml")


    # -----------------------------------------
    # Start MLflow UI in the background
    # -----------------------------------------
    get_ipython().system_raw(
        f"nohup mlflow ui "
        f"--backend-store-uri file:{mlruns_dir} "  # Where mlruns are stored
        f"--port {port} "                          # Local MLflow port
        f"--host 0.0.0.0 "                         # Expose it for ngrok
        f"> /content/mlflow.log 2>&1 &"            # Log output to file
    )

    time.sleep(3)  # Wait for MLflow to start


    # -----------------------------------------
    # FIX 2: Use host_header="rewrite"
    # Required to make MLflow UI accessible via ngrok
    # -----------------------------------------
    public_url = ngrok.connect(port, "http", host_header="rewrite")


    # -----------------------------------------
    # Return public MLflow tracking URL
    # -----------------------------------------
    print("Your MLflow Tracking URL:", public_url)

    return public_url

In [ ]:
import os  # File/folder operations (join paths, check existence)
import pytorch_lightning as pl  # Lightning framework for structured training
from torch.utils.data import DataLoader  # Wraps datasets into batches with shuffle & workers
from torchvision import datasets, transforms  # Dataset loading & image preprocessing

# ----------------------------------------
# LightningDataModule for Plant Disease Dataset
# ----------------------------------------
class PlantDataModule(pl.LightningDataModule):

    def __init__(self, data_dir, batch_size=32, img_size=224):
        """
        Initialize the DataModule.
        Args:
            data_dir (str): Root path to dataset
            batch_size (int): Images per batch
            img_size (int): Resize images to square size
        """
        super().__init__()  # Initialize LightningDataModule
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.img_size = img_size

        # -------------------------------
        # Transformations for training images
        # -------------------------------
        self.train_tfms = transforms.Compose([
            transforms.Resize((img_size, img_size)),
            transforms.RandomResizedCrop(img_size, scale=(0.8, 1.0)),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(15),
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                               std=[0.229, 0.224, 0.225])                     # Convert PIL Image → PyTorch Tensor
        ])

        # -------------------------------
        # Transformations for validation/test images
        # -------------------------------
        self.val_tfms = transforms.Compose([
            transforms.Resize((img_size, img_size)),  # Resize all images
            transforms.ToTensor()                     # Convert PIL Image → PyTorch Tensor
        ])

    # -------------------------------
    # Setup datasets (train/val/test)
    # -------------------------------
    def setup(self, stage=None):
        """
        Called by Lightning at the start of training/validation/testing.
        Loads datasets using ImageFolder.
        """
        train_dir = os.path.join(self.data_dir, "Train")  # Path to training images
        val_dir = os.path.join(self.data_dir, "Valid")    # Path to validation images
        test_dir = os.path.join(self.data_dir, "Test")    # Path to test images

        # Safety check: training folder must exist
        if not os.path.exists(train_dir):
            raise FileNotFoundError(f"Expected train folder at {train_dir}")

        # Load datasets
        self.train_ds = datasets.ImageFolder(train_dir, transform=self.train_tfms)
        self.val_ds = datasets.ImageFolder(val_dir, transform=self.val_tfms) if os.path.exists(val_dir) else None
        self.test_ds = datasets.ImageFolder(test_dir, transform=self.val_tfms) if os.path.exists(test_dir) else None

    # -------------------------------
    # Training DataLoader
    # -------------------------------
    def train_dataloader(self):
        return DataLoader(
            self.train_ds,
            batch_size=self.batch_size,
            shuffle=True,  # Shuffle each epoch for better generalization
            num_workers=2  # Parallel data loading
        )

    # -------------------------------
    # Validation DataLoader
    # -------------------------------
    def val_dataloader(self):
        return DataLoader(
            self.val_ds,
            batch_size=self.batch_size,
            num_workers=2
        ) if self.val_ds else None  # Returns None if no validation dataset

    # -------------------------------
    # Test DataLoader
    # -------------------------------
    def test_dataloader(self):
        return DataLoader(
            self.test_ds,
            batch_size=self.batch_size,
            num_workers=2
        ) if self.test_ds else None  # Returns None if no test dataset

In [ ]:
# =====================================================
# 🌱 PlantDiseaseModel — PyTorch Lightning Module
# =====================================================

import pytorch_lightning as pl  # Lightning framework for structured training
import torch                     # Core PyTorch library
import timm                      # Library with SOTA pretrained vision models

# -------------------------------
# Learning rate
# -------------------------------

# -------------------------------
# LightningModule for Plant Disease Classification
# -------------------------------
class PlantDiseaseModel(pl.LightningModule):
    def __init__(self, model_name="efficientnet_b0", lr=1e-4, num_classes=3):
        """
        Plant Disease Classification Model using timm pretrained backbone

        Args:
            model_name (str): timm model identifier (e.g. 'efficientnet_b0', 'resnet50', 'convnext_tiny')
            lr (float): initial learning rate
            num_classes (int): number of plant disease classes
        """
        super().__init__()
        self.save_hyperparameters()

        # Model backbone
        self.backbone = timm.create_model(
            model_name,
            pretrained=True,
            num_classes=num_classes,          # replaces classifier head
            drop_rate=0.3                     # light dropout in classifier
        )

        # Loss function
        self.loss_fn = torch.nn.CrossEntropyLoss(label_smoothing=0.1)  # <-- small label smoothing often helps

        # For manual collection → later plotting
        self.train_losses = []
        self.train_accs   = []
        self.val_losses   = []
        self.val_accs     = []

        # We'll also keep epoch-level averages for cleaner plots
        self.train_loss_epoch = []
        self.train_acc_epoch  = []
        self.val_loss_epoch   = []
        self.val_acc_epoch    = []

    def forward(self, x):
        return self.backbone(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.loss_fn(logits, y)

        # Accuracy
        preds = logits.argmax(dim=1)
        acc = (preds == y).float().mean()

        # Logging (per step / batch)
        self.log("train_loss", loss, prog_bar=True, sync_dist=True)
        self.log("train_acc",  acc,  prog_bar=True, sync_dist=True)

        # Collect for plotting
        self.train_losses.append(loss.item())
        self.train_accs.append(acc.item())

        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.loss_fn(logits, y)

        preds = logits.argmax(dim=1)
        acc = (preds == y).float().mean()

        self.log("val_loss", loss, prog_bar=True, sync_dist=True)
        self.log("val_acc",  acc,  prog_bar=True, sync_dist=True)

        self.val_losses.append(loss.item())
        self.val_accs.append(acc.item())

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.loss_fn(logits, y)

        preds = logits.argmax(dim=1)
        acc = (preds == y).float().mean()

        self.log("test_loss", loss, prog_bar=True)
        self.log("test_acc",  acc,  prog_bar=True)

    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        """Used with trainer.predict() — returns probabilities or logits"""
        x, _ = batch
        logits = self(x)
        return logits.softmax(dim=1)   # ← usually better to return probabilities

    # ────────────────────────────────────────────────
    #  Epoch-end hooks → compute & store average metrics
    # ────────────────────────────────────────────────
    def on_train_epoch_end(self):
        if len(self.train_losses) > 0:
            avg_loss = sum(self.train_losses) / len(self.train_losses)
            avg_acc  = sum(self.train_accs)  / len(self.train_accs)

            self.train_loss_epoch.append(avg_loss)
            self.train_acc_epoch.append(avg_acc)

            # Optional: print or log epoch average
            self.log("train_loss_epoch", avg_loss, prog_bar=False)
            self.log("train_acc_epoch",  avg_acc,  prog_bar=False)

            # Clear batch-wise lists (optional - depends if you want all history)
            # self.train_losses.clear()
            # self.train_accs.clear()

    def on_validation_epoch_end(self):
        if len(self.val_losses) > 0:
            avg_loss = sum(self.val_losses) / len(self.val_losses)
            avg_acc  = sum(self.val_accs)  / len(self.val_accs)

            self.val_loss_epoch.append(avg_loss)
            self.val_acc_epoch.append(avg_acc)

            self.log("val_loss_epoch", avg_loss, prog_bar=False)
            self.log("val_acc_epoch",  avg_acc,  prog_bar=False)

            # self.val_losses.clear()
            # self.val_accs.clear()
    # -------------------------------
    # Optimizer configuration
    # -------------------------------
    def configure_optimizers(self):
      optimizer = torch.optim.Adam(
          self.parameters(),
          lr=self.hparams.lr
      )

      scheduler = {
          "scheduler": torch.optim.lr_scheduler.ReduceLROnPlateau(
              optimizer,
              mode="min",      # val_loss কমানো লক্ষ্য
              factor=0.5,      # lr = lr * 0.5
              patience=3       # 3 epoch wait করবে
          ),
          "monitor": "val_loss",
          "interval": "epoch",
          "frequency": 1
      }

      return {
          "optimizer": optimizer,
          "lr_scheduler": scheduler
      }

In [ ]:
print("Preparing data module and model (this will error if your dataset path is wrong)...")
dm = PlantDataModule(EXTRACT_DIR)

try:
    dm.setup()
except Exception as e:
    raise RuntimeError(f"Data setup failed: {e}")

num_classes = len(dm.train_ds.classes)
print("Classes:", dm.train_ds.classes)

model = PlantDiseaseModel(num_classes=num_classes)

In [ ]:
checkpoint_path = os.path.join(
    os.getcwd(), "checkpoints", "/content/drive/MyDrive/BongoDev/Project-1/model.pth"
)

In [ ]:
if os.path.exists(checkpoint_path):
    model.load_state_dict(torch.load(checkpoint_path))
    print("Model loaded successfully from checkpoints.")

In [ ]:
trainer = pl.Trainer(
    max_epochs=15,
)

trainer.fit(model, dm)

In [ ]:
# SAVE MODEL HERE
MODEL_PATH = "/content/drive/MyDrive/BongoDev/Project-1/model.pth"
# Use the model's state_dict() for saving the raw weights
torch.save(model.state_dict(), MODEL_PATH)
print("Model saved at:", MODEL_PATH)

num_classes = len(dm.train_ds.classes)
num_classes

In [ ]:
import matplotlib.pyplot as plt

# Assuming you collected per-epoch averages — or just plot what you have
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(model.train_losses, label='Train loss (per batch)', alpha=0.6)
plt.plot(model.val_losses,   label='Val loss', marker='o')
plt.title("Loss Curves")
plt.xlabel("Batch / Epoch")
plt.ylabel("CrossEntropy Loss")
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(model.train_accs, label='Train acc (per batch)', alpha=0.6)
plt.plot(model.val_accs,   label='Val acc', marker='o')
plt.title("Accuracy Curves")
plt.xlabel("Batch / Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
import torch
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns
import matplotlib.pyplot as plt

model.eval()
model.to('cuda' if torch.cuda.is_available() else 'cpu')

all_preds = []
all_labels = []

test_dl = dm.test_dataloader()
if test_dl is None:
    print("No test set found → using validation set")
    test_dl = dm.val_dataloader()

with torch.no_grad():
    for batch in test_dl:
        x, y = batch
        x = x.to(model.device)
        logits = model(x)
        preds = logits.argmax(dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(y.cpu().numpy())

# Create & plot
cm = confusion_matrix(all_labels, all_preds)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=dm.train_ds.classes)

plt.figure(figsize=(10, 8))
disp.plot(cmap='Blues', xticks_rotation=45)
plt.title("Confusion Matrix (Test set)")
plt.tight_layout()
plt.show()

# Or with seaborn (nicer for many classes)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=dm.train_ds.classes,
            yticklabels=dm.train_ds.classes)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
!pip install grad-cam

In [ ]:
# ────────────────────────────────────────────────────────────────
#  Requirements (run once if needed)
# ────────────────────────────────────────────────────────────────
# !pip install grad-cam -q

import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import torch
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image

# ────────────────────────────────────────────────────────────────
#  Configuration
# ────────────────────────────────────────────────────────────────

image_paths = [
    "/content/drive/MyDrive/BongoDev/Project-1/na1.jpg",
    "/content/drive/MyDrive/BongoDev/Project-1/na.JPG",
    "/content/drive/MyDrive/BongoDev/Project-1/na2.JPG",
]

# Suggested labels — change them based on what you expect / actual images
image_labels = ["Sample 1", "Sample 2", "Sample 3"]

# Target layer — try this first; if error occurs, switch to [model.backbone.features[-1]]
target_layers = [model.backbone.conv_head]

# ────────────────────────────────────────────────────────────────
#  Prepare Grad-CAM
# ────────────────────────────────────────────────────────────────
device = next(model.parameters()).device
model.eval()

cam = GradCAM(model=model, target_layers=target_layers)
val_transform = dm.val_tfms

# ────────────────────────────────────────────────────────────────
#  Visualization with arrows pointing to most important region
# ────────────────────────────────────────────────────────────────
fig, axes = plt.subplots(1, 3, figsize=(16, 6), dpi=130)
fig.suptitle("Grad-CAM: Where the Model is Looking (Arrow = Strongest Attention)",
             fontsize=16, fontweight='bold', y=1.04)

for idx, (img_path, label) in enumerate(zip(image_paths, image_labels)):
    try:
        # Load image
        img_pil = Image.open(img_path).convert("RGB")
        img_tensor = val_transform(img_pil).unsqueeze(0).to(device)

        # Predict
        with torch.no_grad():
            logits = model(img_tensor)
            pred_class_idx = logits.argmax(dim=1).item()
            pred_class_name = dm.train_ds.classes[pred_class_idx]
            confidence = torch.softmax(logits, dim=1)[0, pred_class_idx].item()

        # Grad-CAM
        targets = [ClassifierOutputTarget(pred_class_idx)]
        grayscale_cam = cam(input_tensor=img_tensor, targets=targets)
        grayscale_cam = grayscale_cam[0, :]

        # Overlay
        img_np = np.array(img_pil.resize((224, 224))) / 255.0
        visualization = show_cam_on_image(img_np, grayscale_cam, use_rgb=True)

        # ─── Find the position of maximum activation ───
        h, w = grayscale_cam.shape
        max_y, max_x = np.unravel_index(np.argmax(grayscale_cam), grayscale_cam.shape)

        # Convert to image coordinates (0–223 range)
        arrow_x = max_x
        arrow_y = max_y

        # Plot image + heatmap overlay
        ax = axes[idx]
        ax.imshow(visualization)

        # ─── Draw arrow pointing to hottest spot ───
        ax.annotate(
            "",
            xy=(arrow_x, arrow_y),               # where arrow points TO
            xytext=(arrow_x + 60, arrow_y - 60), # where arrow starts FROM (tail)
            arrowprops=dict(
                facecolor='red',
                edgecolor='darkred',
                width=4,           # thick line
                headwidth=14,      # big arrow head
                headlength=18,
                shrink=0.08,       # slight shrink so tip touches exactly
                alpha=0.9,
                linewidth=1.5,
                zorder=10
            )
        )

        # Add small circle at the exact hotspot (helps visibility)
        ax.plot(arrow_x, arrow_y, marker='o', markersize=14,
                color='red', markeredgecolor='white', markeredgewidth=2.5, zorder=11)

        # Title with prediction info
        ax.set_title(f"{label}\nPred: {pred_class_name}\nConf: {confidence:.1%}",
                     fontsize=11, pad=12, fontweight='medium')
        ax.axis('off')

    except Exception as e:
        ax = axes[idx]
        ax.text(0.5, 0.5, f"Error\n{str(e)[:60]}...",
                ha='center', va='center', color='red', fontsize=10)
        ax.axis('off')

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()